In [1]:
from enum import Enum


# RISCV-32I 6 种类型
class OpCode(Enum):
    R = "0110011"
    I_JALR = "1100111"
    I_CALC = "0010011"
    I_LOAD = "0000011"
    S = "0100011"
    B = "1100011"
    U_LUI = "0110111"
    U_AUIPC = "0010111"
    J = "1101111"


# 部分 I 型指令
class I_CALCFunct3(Enum):
    ADDI = "000"
    SLTI = "010"
    SLTIU = "011"
    XORI = "100"
    ORI = "110"
    ANDI = "111"
    SLLI = "001"
    SRLI = "101"
    SRAI = "101"


class I_JALRFunct3(Enum):
    JALR = "000"


class I_LOADFunct3(Enum):
    LB = "000"
    LH = "001"
    LW = "010"
    LBU = "100"
    LHU = "101"


# 部分 S 型指令
class SFunct3(Enum):
    SB = "000"
    SH = "001"
    SW = "010"


# B 型指令
class BFunct3(Enum):
    BEQ = "000"
    BNE = "001"
    BLT = "100"
    BGE = "101"
    BLTU = "110"
    BGEU = "111"


# 部分 R 型指令
class RFunct3(Enum):
    ADD = "000"
    SUB = "000"
    SLL = "001"
    SLT = "010"
    SLTU = "011"
    XOR = "100"
    SRL = "101"
    SRA = "101"
    OR = "110"
    AND = "111"


class RFunct7(Enum):
    ADD = "0000000"
    SUB = "0100000"
    SRA = "0100000"


class InstructionInfo:
    opcode: OpCode
    rs1: int
    rs2: int
    rd: int
    funct3: Enum
    funct7: Enum
    imm: int


class Component:
    """
    元件基类, 所有其他处理器设计元件都需要继承此类

    用于性能记录
    """

    def __init__(self) -> None:
        self.class_name = self.__class__.__name__

        self.is_locked = False
        self.is_empty = True

    def lock(self):
        if self.is_locked:
            print(f"{self.class_name} is locked")
            exit(1)
        self.is_locked = True

    def unlock(self):
        if not self.is_locked:
            print(f"try to unlock a not locked lock in {self.class_name}")
            exit(1)
        self.is_locked = False


class ALUop(Enum):
    ADD = 0b0000
    SUB = 0b1000
    LSHIFT = 0b0001
    # LSHIFT_ = 0b1001
    OUTPUT_B = 0b0011  # 选择 B 结果直接输出
    # B_ = 0b1011
    XOR = 0b0100
    # XOR_ = 0b1100
    LRSHIFT = 0b0101
    ARSHIFT = 0b1101
    OR = 0b0110
    # OR_ = 0b1110
    AND = 0b0111
    # AND_ = 0b1111


class ALU_Asrc(Enum):
    RA = 0
    PC = 1


class ALU_Bsrc(Enum):
    RB = 0
    IMM = 1
    NEXT = 2  # pc + 4


class PCsrc(Enum):
    PC = 0
    IMM = 1
    JAL = 2
    JALR = 3
    AUIPC = 4


class MemtoReg(Enum):
    READ_DATA = 0
    ALU_RESULT = 1


class MemOp(Enum):
    NONE = 0b111
    SIGN1 = 0b000
    SIGN2 = 0b001
    SIGN4 = 0b010
    UNSIGN1 = 0b100
    UNSIGN2 = 0b101


class ControlSignal:
    """
    控制信号, 决策对应 MUX 应该选择使用哪一个作为输入
    """

    ALU_Asrc: ALU_Asrc # ALU 的第一个输入选择哪一个
    ALU_Bsrc: ALU_Bsrc  # ALU 的第二个输入选择哪一个
    ALUop: ALUop  # ALU 如何进行计算
    RegWrite: bool  # 是否写寄存器
    MemRead: bool  # 是否读内存
    MemWrite: bool  # 是否写内存
    MemtoReg: MemtoReg  # 写回寄存器的值选择哪一个
    MemOp: MemOp  # 读取内存的方式
    PCsrc: PCsrc  # 如何更新 PC


class IF_ID(Component):
    instruction: str
    pc: int


class ID_EX(Component):
    ra: int
    rb: int
    rd: int
    imm: int
    ctl_sig: ControlSignal
    # 条件跳转的方式, 只对于 B 指令在 EX 阶段使用
    Branch: BFunct3

    # 存 rs1 和 rs2 是为了在 EX 阶段检测数据冒险
    rs1: int
    rs2: int
    pc: int


class EX_MEM(Component):
    alu_result: int
    rb: int
    rd: int
    MemRead: bool
    MemWrite: bool
    MemtoReg: MemtoReg
    MemOp: MemOp
    RegWrite: bool
    
    # 条件跳转是否成立
    branch_cond: bool
    PCsrc: PCsrc
    pc: int
    imm: int

class MEM_WB(Component):
    rd: int
    read_data: int
    alu_result: int
    MemtoReg: MemtoReg
    RegWrite: bool


In [2]:

class Instruction:
    def __init__(self) -> None:
        self.name = self.__class__.__name__
        # print(f"{self.name}")
        self.control_signal = ControlSignal()

    def get_control_signal(self):
        raise NotImplementedError(f"{self.name} should overwrite get_control_signal")


class R_ADD(Instruction):
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class R_SUB(Instruction):
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.SUB
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class R_SLL(Instruction):
    ...


class R_SLT(Instruction):
    ...


class R_SLTU(Instruction):
    ...


class R_XOR(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.XOR
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class R_SRL(Instruction):
    ...


class R_SRA(Instruction):
    ...


class R_OR(Instruction):
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.OR
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class R_AND(Instruction):
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.AND
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class I_ADDI(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class I_SLTI(Instruction):
    ...


class I_SLTIU(Instruction):
    ...


class I_XORI(Instruction):
    ...


class I_ORI(Instruction):
    ...


class I_ANDI(Instruction):
    ...


class I_SLLI(Instruction):
    ...


class I_SRLI(Instruction):
    ...


class I_SRAI(Instruction):
    ...


class I_JALR(Instruction):
    
    def get_control_signal(self):
        
        self.control_signal.ALU_Asrc = ALU_Asrc.PC
        self.control_signal.ALU_Bsrc = ALU_Bsrc.NEXT
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.JALR
        return self.control_signal


class I_LB(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = True
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.READ_DATA
        self.control_signal.MemOp = MemOp.SIGN1
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class I_LH(Instruction):
    ...


class I_LW(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = True
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.READ_DATA
        self.control_signal.MemOp = MemOp.SIGN4
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class I_LBU(Instruction):
    ...


class I_LHU(Instruction):
    ...


class S_SB(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = False
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = True
        self.control_signal.MemtoReg = MemtoReg.READ_DATA
        self.control_signal.MemOp = MemOp.SIGN1
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class S_SH(Instruction):
    ...


class S_SW(Instruction):
    def get_control_signal(self) -> ControlSignal:
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = False
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = True
        self.control_signal.MemtoReg = MemtoReg.READ_DATA
        self.control_signal.MemOp = MemOp.SIGN4
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal


class B_BEQ(Instruction):
    ...


class B_BNE(Instruction):
    def get_control_signal(self):
        # 这里的 ALUsrc 是 RB 而不是 IMM 是因为优化了流水线结构
        # 在 ID 阶段之后添加了 branch_ALU 用于比较计算
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.RB
        self.control_signal.ALUop = ALUop.SUB
        self.control_signal.RegWrite = False
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.READ_DATA
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.IMM
        return self.control_signal


class B_BLT(Instruction):
    ...


class B_BGE(Instruction):
    ...


class B_BLTU(Instruction):
    ...


class B_BGEU(Instruction):
    ...


class U_LUI(Instruction):
    
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.RA
        self.control_signal.ALU_Bsrc = ALU_Bsrc.IMM
        self.control_signal.ALUop = ALUop.OUTPUT_B
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.PC
        return self.control_signal

class U_AUIPC(Instruction):
    ...


class J_JAL(Instruction):
    def get_control_signal(self):
        self.control_signal.ALU_Asrc = ALU_Asrc.PC
        self.control_signal.ALU_Bsrc = ALU_Bsrc.NEXT
        self.control_signal.ALUop = ALUop.ADD
        self.control_signal.RegWrite = True
        self.control_signal.MemRead = False
        self.control_signal.MemWrite = False
        self.control_signal.MemtoReg = MemtoReg.ALU_RESULT
        self.control_signal.MemOp = MemOp.NONE
        self.control_signal.PCsrc = PCsrc.IMM
        return self.control_signal


In [3]:
import copy


class IR(Component):
    """
    中间寄存器, 用于存储流水线 5 阶段的 4 个中间阶段的执行结果

    ISA 执行 run 时会从前一阶段的 IF_ID 取值, 并将结果写入到下一阶段的 pre_ID_EXE 中, 以此类推
    全部执行结束后会调用 update 将所有 pre_IR 的结果更新到 IR 中

    相当于采用了双倍的硬件/空间
    """

    def __init__(self) -> None:
        super().__init__()
        self.IF_ID = IF_ID()
        self.ID_EX = ID_EX()
        self.EX_MEM = EX_MEM()
        self.MEM_WB = MEM_WB()

        self.pre_IF_ID = IF_ID()
        self.pre_ID_EX = ID_EX()
        self.pre_EX_MEM = EX_MEM()
        self.pre_MEM_WB = MEM_WB()

        self.bubble = False

    def update(self):
        if self.bubble:
            self.EX_MEM.is_empty = True
            self.MEM_WB = copy.deepcopy(self.pre_MEM_WB)
        else:
            self.IF_ID = copy.deepcopy(self.pre_IF_ID)
            self.ID_EX = copy.deepcopy(self.pre_ID_EX)
            self.EX_MEM = copy.deepcopy(self.pre_EX_MEM)
            self.MEM_WB = copy.deepcopy(self.pre_MEM_WB)

        self.bubble = False

    def is_flushed(self):
        """
        当 pre_IF_ID 取指为空, 且所有阶段的 IR 均空, 终止流水线
        """
        return (
            self.pre_IF_ID.is_empty
            and self.IF_ID.is_empty
            and self.ID_EX.is_empty
            and self.EX_MEM.is_empty
            and self.MEM_WB.is_empty
        )

    def reset(self):
        self.IF_ID = IF_ID()
        self.ID_EX = ID_EX()
        self.EX_MEM = EX_MEM()
        self.MEM_WB = MEM_WB()

        self.pre_IF_ID = IF_ID()
        self.pre_ID_EX = ID_EX()
        self.pre_EX_MEM = EX_MEM()
        self.pre_MEM_WB = MEM_WB()

        self.bubble = False


class RegisterGroup(Component):
    def __init__(self, number=32) -> None:
        super().__init__()
        self.registers = [0] * number

    def read(self, rs1: int, rs2: int) -> (int, int):
        """
        读取对应寄存器的值
        """
        self.lock()
        ra = None
        rb = None
        if rs1 is not None:
            ra = self.registers[rs1]
        if rs2 is not None:
            rb = self.registers[rs2]
        self.unlock()
        return ra, rb

    def write(self, rd: int, write_data: int, RegWrite: bool):
        """
        只有当 RegWrite 信号为 1 才可以写入寄存器
        """
        if RegWrite is True:
            self.lock()
            self.registers[rd] = write_data
            self.unlock()

    def reset(self):
        """
        重置寄存器状态, 归零
        """
        for i in range(len(self.registers)):
            self.registers[i] = 0

    def __setitem__(self, index, value):
        self.registers[index] = value

    def __getitem__(self, index):
        return self.registers[index]


class Memory(Component):
    def __init__(self, addr_range=0x200) -> None:
        super().__init__()
        self.memory = [0] * addr_range

    def read(self, addr: int, MemRead: bool, op: Enum):
        if MemRead is True:
            self.lock()
            if op == MemOp.SIGN1:
                value = self.memory[addr]
            elif op == MemOp.SIGN2:
                value = self.memory[addr]
                value += self.memory[addr + 1] << 8
            elif op == MemOp.SIGN4:
                value = self.memory[addr]
                value += self.memory[addr + 1] << 8
                value += self.memory[addr + 2] << 16
                value += self.memory[addr + 3] << 24
            else:
                raise ValueError()
            self.unlock()
            return value

    def write(self, addr: int, write_data: int, MemWrite: bool, op: Enum):
        if MemWrite is True:
            self.lock()
            if op == MemOp.SIGN1:
                self.memory[addr] = write_data
            elif op == MemOp.SIGN2:
                self.memory[addr] = write_data & 0xFF
                self.memory[addr + 1] = write_data & 0xFF00
            elif op == MemOp.SIGN4:
                self.memory[addr] = write_data & 0xFF
                self.memory[addr + 1] = write_data & 0xFF00
                self.memory[addr + 2] = write_data & 0xFF0000
                self.memory[addr + 3] = write_data & 0xFF000000
            else:
                raise ValueError()
            self.unlock()

    def reset(self):
        """
        重置内存状态, 归零
        """
        for i in range(len(self.memory)):
            self.memory[i] = 0

    def __setitem__(self, index, value):
        self.memory[index] = value

    def __getitem__(self, index):
        return self.memory[index]


class ALU(Component):
    def __init__(self, op: Enum = None) -> None:
        super().__init__()
        # 设置 op 表示某一类特定的功能
        # 比如 ID_adder 只做加法
        self.op = op

    def calc(self, input_a: int, input_b: int, op: Enum = None) -> int:
        if op is None:
            op = self.op
        if op == ALUop.ADD:
            return input_a + input_b
        elif op == ALUop.SUB:
            return input_a - input_b
        elif op == ALUop.LSHIFT:
            return input_a << input_b
        elif op == ALUop.OUTPUT_B:
            return input_b
        elif op == ALUop.XOR:
            return input_a ^ input_b
        elif op == ALUop.LRSHIFT:
            return (input_a & 0xFFFFFFFF) >> input_b
        elif op == ALUop.ARSHIFT:
            return input_a >> input_b
        elif op == ALUop.OR:
            return input_a | input_b
        elif op == ALUop.AND:
            return input_a & input_b
        else:
            raise ValueError(f"unknown ALU operator value {op}")


class PipelineISA:
    """
    流水线处理器架构

    正常来说 5 阶段流水线是同步执行的, 通用寄存器组和存储器在时钟上升沿写入,IR和中间寄存器在时钟下降沿写入
    考虑到 Python 没有办法做到电路级模拟, 理论上来说如果想要实现时序级模拟, 需要使用 5 个线程和 5 把锁,
    以保证当前阶段写入IR和中间寄存器之前该寄存器的值已经被下一个阶段读取

    但实际上可以利用 IR 作为中间变量, 从 IF_ID 中读取, 写入到 pre_ID_EX 中, 最后同步更新所有的 IR,
    这样就实现了采用串行的方式模拟流水线
    """

    def __init__(self) -> None:
        # 基础配置信息
        register_number = 32
        addr_range = 0x2000

        self.pc = 0
        self.registers = RegisterGroup(register_number)  # 寄存器组
        self.memory = Memory(addr_range)  # 内存
        self.ALU = ALU()
        self.IR = IR()

        self.ID_adder = ALU(ALUop.ADD)  # 加法器, 用于计算 ID 阶段的 PC + IMM
        self.ID_subtractor = ALU(ALUop.SUB)  # 减法器, 用于计算比较

    def load_instructions(self, instructions, pc=0x100):
        self.pc = pc
        # 小端存储
        for inst in instructions:
            instruction_str = format(inst, "032b")
            self.memory[pc + 3] = int(instruction_str[:8], 2)
            self.memory[pc + 2] = int(instruction_str[8:16], 2)
            self.memory[pc + 1] = int(instruction_str[16:24], 2)
            self.memory[pc] = int(instruction_str[24:], 2)
            pc += 4

    def reset(self):
        """
        重置处理器状态, 用于多次运行
        """
        self.pc = 0
        self.registers.reset()
        self.memory.reset()
        self.IR.reset()

    def run(self):
        self.step = 0
        while True:
            self.step += 1
            # stage_if ~ stage_wb 阶段的写入IR并不是真正的写入
            # 此时的才是真正的更新流水线阶段需要使用的 IR 的值
            self.IR.update()

            # 优先写回
            self.stage_wb()
            self.stage_if()
            # 当取指之后, 流水线 IR 全空且新指令也为 0, 则说明已经流水线全部结束且没有新指令了
            if self.IR.is_flushed():
                break
            self.stage_id()
            self.stage_ex()
            self.stage_mem()
            self.update_pc()
            # self.show_info()

    def stage_if(self):
        """
        IF-取指令.根据PC中的地址在指令存储器中取出一条指令
        """
        # 小端取数
        self.IR.pre_IF_ID.instruction = ""
        self.IR.pre_IF_ID.instruction += format(self.memory[self.pc + 3], "08b")
        self.IR.pre_IF_ID.instruction += format(self.memory[self.pc + 2], "08b")
        self.IR.pre_IF_ID.instruction += format(self.memory[self.pc + 1], "08b")
        self.IR.pre_IF_ID.instruction += format(self.memory[self.pc], "08b")

        # 保存当前指令对应 PC 值, 以供 ID 阶段的 B/J 型指令计算跳转地址
        self.IR.pre_IF_ID.pc = self.pc

        if int(self.IR.pre_IF_ID.instruction) == 0:
            self.IR.pre_IF_ID.is_empty = True
        else:
            self.IR.pre_IF_ID.is_empty = False

    def stage_id(self):
        """
        ID-译码 解析指令并读取寄存器的值
        """
        if self.IR.IF_ID.is_empty:
            # IF_ID 的 IR 为空, 跳过 ID 阶段, 并设置下一阶段的 IR 也为空
            self.IR.pre_ID_EX.is_empty = True
            return
        else:
            self.IR.pre_ID_EX.is_empty = False

        instruction = self.IR.IF_ID.instruction
        instruction_info = InstructionInfo()

        opcode_type = OpCode(instruction[-7:])
        instruction_info.opcode = opcode_type
        if opcode_type == OpCode.R:
            instruction_info.funct7 = RFunct7(instruction[:7])
            instruction_info.rs2 = int(instruction[7:12], 2)
            instruction_info.rs1 = int(instruction[12:17], 2)
            instruction_info.funct3 = RFunct3(instruction[17:20])
            instruction_info.rd = int(instruction[20:25], 2)
            instruction_info.imm = None
        elif opcode_type in (OpCode.I_LOAD, OpCode.I_CALC, OpCode.I_JALR):
            instruction_info.funct7 = None
            instruction_info.rs2 = None
            instruction_info.rs1 = int(instruction[12:17], 2)
            funct3 = {
                OpCode.I_LOAD: I_LOADFunct3,
                OpCode.I_CALC: I_CALCFunct3,
                OpCode.I_JALR: I_JALRFunct3,
            }
            instruction_info.funct3 = funct3[opcode_type](instruction[17:20])
            instruction_info.rd = int(instruction[20:25], 2)
            instruction_info.imm = self.binary_str(instruction[:12])

        elif opcode_type == OpCode.S:
            instruction_info.funct7 = None
            instruction_info.rs2 = int(instruction[7:12], 2)
            instruction_info.rs1 = int(instruction[12:17], 2)
            instruction_info.funct3 = SFunct3(instruction[17:20])
            instruction_info.rd = None
            instruction_info.imm = self.binary_str(instruction[:7] + instruction[20:25])
        elif opcode_type == OpCode.B:
            instruction_info.funct7 = None
            instruction_info.rs2 = int(instruction[7:12], 2)
            instruction_info.rs1 = int(instruction[12:17], 2)
            instruction_info.funct3 = BFunct3(instruction[17:20])
            instruction_info.rd = None
            instruction_info.imm = self.binary_str(
                instruction[0] + instruction[24] + instruction[1:7] + instruction[20:24] + "0"
            )

        elif opcode_type in (OpCode.U_AUIPC, OpCode.U_LUI):
            instruction_info.funct7 = None
            instruction_info.rs2 = None
            instruction_info.rs1 = None
            instruction_info.funct3 = None
            instruction_info.rd = int(instruction[20:25], 2)
            instruction_info.imm = self.binary_str(instruction[:20] + "0" * 12)
        elif opcode_type == OpCode.J:
            instruction_info.funct7 = None
            instruction_info.rs2 = None
            instruction_info.rs1 = None
            instruction_info.funct3 = None
            instruction_info.rd = int(instruction[20:25], 2)
            instruction_info.imm = self.binary_str(
                instruction[0] + instruction[12:20] + instruction[11] + instruction[1:11] + "0"
            )
        else:
            raise ValueError("unknown opcode type")

        self.IR.pre_ID_EX.ra, self.IR.pre_ID_EX.rb = self.registers.read(
            instruction_info.rs1, instruction_info.rs2
        )
        self.IR.pre_ID_EX.rd = instruction_info.rd
        self.IR.pre_ID_EX.imm = instruction_info.imm
        self.IR.pre_ID_EX.ctl_sig = self.get_control_signal(instruction_info)

        self.IR.pre_ID_EX.rs1 = instruction_info.rs1
        self.IR.pre_ID_EX.rs2 = instruction_info.rs2
        self.IR.pre_ID_EX.pc = self.IR.IF_ID.pc

        if opcode_type == OpCode.B:
            # B 类型单独设置 Branch 标记位用于 update_pc 阶段计算
            # 需要根据 Branch 的类型来   判断跳转条件是否成立
            self.IR.pre_ID_EX.Branch = instruction_info.funct3
        else:
            self.IR.pre_ID_EX.Branch = None

    def get_control_signal(self, instruction_info: InstructionInfo) -> ControlSignal:
        RISCV_32I_instructions = {
            OpCode.R: {
                RFunct3.ADD: R_ADD,
                # RFunct3.SUB: R_SUB,
                RFunct3.SLL: R_SLL,
                RFunct3.SLT: R_SLT,
                RFunct3.SLTU: R_SLTU,
                RFunct3.XOR: R_XOR,
                RFunct3.SRL: R_SRL,
                RFunct3.SRA: R_SRA,
                RFunct3.OR: R_OR,
                RFunct3.AND: R_AND,
            },
            OpCode.I_CALC: {
                I_CALCFunct3.ADDI: I_ADDI,
                I_CALCFunct3.SLTI: I_SLTI,
                I_CALCFunct3.SLTIU: I_SLTIU,
                I_CALCFunct3.XORI: I_XORI,
                I_CALCFunct3.ORI: I_ORI,
                I_CALCFunct3.ANDI: I_ANDI,
                I_CALCFunct3.SLLI: I_SLLI,
                I_CALCFunct3.SRLI: I_SRLI,
                I_CALCFunct3.SRAI: I_SRAI,
            },
            OpCode.I_JALR: {I_JALRFunct3.JALR: I_JALR},
            OpCode.I_LOAD: {
                I_LOADFunct3.LB: I_LB,
                I_LOADFunct3.LH: I_LH,
                I_LOADFunct3.LW: I_LW,
                I_LOADFunct3.LBU: I_LBU,
                I_LOADFunct3.LHU: I_LHU,
            },
            OpCode.S: {SFunct3.SB: S_SB, SFunct3.SH: S_SH, SFunct3.SW: S_SW},
            OpCode.B: {
                BFunct3.BEQ: B_BEQ,
                BFunct3.BNE: B_BNE,
                BFunct3.BLT: B_BLT,
                BFunct3.BGE: B_BGE,
                BFunct3.BLTU: B_BLTU,
                BFunct3.BGEU: B_BGEU,
            },
        }

        # 一些特殊情况的处理, 单独判断
        if instruction_info.funct3 == RFunct3.ADD and instruction_info.funct7 == RFunct7.SUB:
            # ADD/SUB 的 funct3 相同
            instruction = R_SUB
        elif instruction_info.funct3 == RFunct3.SRL and instruction_info.funct7 == RFunct7.SRA:
            # SRL SRA
            instruction = R_SRA
        elif instruction_info.opcode == OpCode.U_AUIPC:
            instruction = U_AUIPC
        elif instruction_info.opcode == OpCode.U_LUI:
            instruction = U_LUI
        elif instruction_info.opcode == OpCode.J:
            instruction = J_JAL
        else:
            instruction = RISCV_32I_instructions[instruction_info.opcode][instruction_info.funct3]

        return instruction().get_control_signal()

    def stage_ex(self):
        """
        EX-执行.对指令的各种操作数进行运算
        """
        if self.IR.ID_EX.is_empty:
            # ID_EX 的 IR 为空, 跳过 EX 阶段, 并设置下一阶段的 IR 也为空
            self.IR.pre_EX_MEM.is_empty = True
            return
        else:
            self.IR.pre_EX_MEM.is_empty = False

        mux_alu_input_a = {ALU_Asrc.RA: self.IR.ID_EX.ra, ALU_Asrc.PC: self.IR.ID_EX.pc}
        mux_alu_input_b = {
            ALU_Bsrc.RB: self.IR.ID_EX.rb,
            ALU_Bsrc.IMM: self.IR.ID_EX.imm,
            ALU_Bsrc.NEXT: 4,
        }

        input_a = mux_alu_input_a[self.IR.ID_EX.ctl_sig.ALU_Asrc]
        input_b = mux_alu_input_b[self.IR.ID_EX.ctl_sig.ALU_Bsrc]

        # 这里优先赋值 rb, 因为后面的 bypass 可能会修改 rb 的结果
        self.IR.pre_EX_MEM.rb = self.IR.ID_EX.rb

        input_a, input_b = self.detect_data_hazard(input_a, input_b)
        if self.IR.bubble:
            return

        alu_result = self.ALU.calc(input_a=input_a, input_b=input_b, op=self.IR.ID_EX.ctl_sig.ALUop)
        self.IR.pre_EX_MEM.alu_result = alu_result

        self.detect_control_hazard(input_a, input_b, alu_result, self.IR.ID_EX.Branch)
        if self.IR.bubble:
            return

        self.IR.pre_EX_MEM.rd = self.IR.ID_EX.rd
        self.IR.pre_EX_MEM.MemRead = self.IR.ID_EX.ctl_sig.MemRead
        self.IR.pre_EX_MEM.MemWrite = self.IR.ID_EX.ctl_sig.MemWrite
        self.IR.pre_EX_MEM.MemtoReg = self.IR.ID_EX.ctl_sig.MemtoReg
        self.IR.pre_EX_MEM.MemOp = self.IR.ID_EX.ctl_sig.MemOp
        self.IR.pre_EX_MEM.RegWrite = self.IR.ID_EX.ctl_sig.RegWrite
        self.IR.pre_EX_MEM.PCsrc = self.IR.ID_EX.ctl_sig.PCsrc
        self.IR.pre_EX_MEM.pc = self.IR.ID_EX.pc
        self.IR.pre_EX_MEM.imm = self.IR.ID_EX.imm

    def stage_mem(self):
        """
        MEM-存储器访问.将数据写入存储器或从存储器中读出数据

        """
        if self.IR.EX_MEM.is_empty:
            # EX_MEM 的 IR 为空, 跳过 MEM 阶段, 并设置下一阶段的 IR 也为空
            self.IR.pre_MEM_WB.is_empty = True
            return
        else:
            self.IR.pre_MEM_WB.is_empty = False

        addr = self.IR.EX_MEM.alu_result
        # 先写后读
        self.memory.write(
            addr=addr,
            write_data=self.IR.EX_MEM.rb,
            MemWrite=self.IR.EX_MEM.MemWrite,
            op=self.IR.EX_MEM.MemOp,
        )
        self.IR.pre_MEM_WB.read_data = self.memory.read(
            addr=addr, MemRead=self.IR.EX_MEM.MemRead, op=self.IR.EX_MEM.MemOp
        )
        self.IR.pre_MEM_WB.alu_result = self.IR.EX_MEM.alu_result
        self.IR.pre_MEM_WB.rd = self.IR.EX_MEM.rd
        self.IR.pre_MEM_WB.MemtoReg = self.IR.EX_MEM.MemtoReg
        self.IR.pre_MEM_WB.RegWrite = self.IR.EX_MEM.RegWrite

    def stage_wb(self):
        """
        WB-写回.将指令运算结果存入指定的寄存器
        """
        if self.IR.MEM_WB.is_empty:
            # MEM_WB 的 IR 为空, 跳过 WB 阶段
            return
        mux_mem_inputs = {
            MemtoReg.READ_DATA: self.IR.MEM_WB.read_data,
            MemtoReg.ALU_RESULT: self.IR.MEM_WB.alu_result,
        }
        write_data = mux_mem_inputs[self.IR.MEM_WB.MemtoReg]
        self.registers.write(
            rd=self.IR.MEM_WB.rd, write_data=write_data, RegWrite=self.IR.MEM_WB.RegWrite
        )

    def update_pc(self):
        """
        更新 PC 指针
        """
        if self.IR.bubble:
            # 冒泡暂停一个周期
            return
        if self.IR.ID_EX.is_empty:
            # 1. 流水线初期, 还没有 ID 阶段
            # 2. ID 阶段被跳过时
            self.pc += 4
        else:
            # 对于 ID 阶段解析为 B/J 型指令, 直接判断跳转条件和地址
            if (
                not self.IR.EX_MEM.is_empty
                and self.IR.pre_EX_MEM.PCsrc == PCsrc.IMM
                and self.IR.pre_EX_MEM.branch_cond
            ):
                new_pc = self.ID_adder.calc(self.IR.pre_EX_MEM.pc, self.IR.pre_EX_MEM.imm)
            elif self.IR.pre_ID_EX.is_empty:
                new_pc = self.pc + 4
            elif self.IR.pre_ID_EX.ctl_sig.PCsrc == PCsrc.JALR:
                a = self.IR.pre_ID_EX.ra
                # TODO: test
                new_pc = self.ID_adder.calc(a, self.IR.pre_ID_EX.imm)
                self.IR.pre_IF_ID.is_empty = True
            elif self.IR.pre_ID_EX.ctl_sig.PCsrc == PCsrc.JAL:
                new_pc = self.ID_adder.calc(self.IR.IF_ID.pc, self.IR.pre_ID_EX.imm)
                self.IR.pre_IF_ID.is_empty = True
            elif self.IR.pre_ID_EX.ctl_sig.PCsrc == PCsrc.AUIPC:
                new_pc = self.IR.pre_ID_EX.imm
                self.IR.pre_IF_ID.is_empty = True
            elif self.IR.pre_ID_EX.ctl_sig.PCsrc in (PCsrc.PC, PCsrc.IMM):
                new_pc = self.pc + 4
            else:
                raise ValueError(f"unknown PCsrc {self.IR.pre_ID_EX.ctl_sig.PCsrc}")

            self.pc = new_pc

    def detect_data_hazard(self, input_a, input_b):
        """
        检测数据冒险

        优先检查靠后的 MEM_WB, 其次检查数据更新更靠前的 EX_MEM
        """
        if not self.IR.MEM_WB.is_empty and self.IR.MEM_WB.RegWrite is True:
            # 如果有 MemtoReg 则使用 read_data
            # 否则使用 alu_result
            bypass_data = (
                self.IR.MEM_WB.read_data
                if self.IR.MEM_WB.MemtoReg is MemtoReg.READ_DATA
                else self.IR.MEM_WB.alu_result
            )
            if self.IR.ID_EX.rs1 == self.IR.MEM_WB.rd:
                input_a = bypass_data
            if self.IR.ID_EX.rs2 == self.IR.MEM_WB.rd:
                input_b = bypass_data

        if not self.IR.EX_MEM.is_empty and self.IR.EX_MEM.RegWrite:
            if self.IR.EX_MEM.MemRead and self.IR.EX_MEM.rd in (
                self.IR.ID_EX.rs1,
                self.IR.ID_EX.rs2,
            ):
                # 要读内存到寄存器的 I_LOAD 指令 lb lw
                # 必须空一个周期
                self.IR.bubble = True
            else:
                # 一些要写寄存器的 R 和 I 型指令
                # 直接 bypass 过去
                if self.IR.ID_EX.rs1 == self.IR.EX_MEM.rd:
                    input_a = self.IR.EX_MEM.alu_result
                if self.IR.ID_EX.rs2 == self.IR.EX_MEM.rd:
                    self.IR.pre_EX_MEM.rb = self.IR.EX_MEM.alu_result
                    if self.IR.ID_EX.ctl_sig.ALU_Bsrc == ALU_Bsrc.RB:
                        input_b = self.IR.EX_MEM.alu_result

        return input_a, input_b

    def detect_control_hazard(self, a, b, alu_result, branch_type: BFunct3):
        """
        B 型指令的控制冒险检测

        J 型指令已经在 ID 阶段提前计算
        """
        if branch_type is None:
            self.IR.pre_EX_MEM.branch_cond = False
            return
        # ZF:零标志位(如果结果等于0,则设置为1,否则设置为0)
        zf = 1 if alu_result == 0 else 0
        # SF:符号标志位(如果结果为负数,则设置为1,否则设置为0)
        sf = 1 if alu_result < 0 else 0
        # OF:溢出标志位(如果结果溢出,则设置为1,否则设置为0)
        of = (
            1 if (a > 0 and b < 0 and alu_result < 0) or (a < 0 and b > 0 and alu_result > 0) else 0
        )
        # CF:进位标志位(如果a小于b,则设置为1,否则设置为0)
        cf = 1 if a < b else 0
        if self.check_branch_condition(branch_type, zf, sf, of, cf):
            self.IR.pre_EX_MEM.branch_cond = True
            # 清空 IF_ID ID_EX 的 IR 内容
            self.IR.pre_IF_ID.is_empty = True
            self.IR.pre_ID_EX.is_empty = True
        else:
            self.IR.pre_EX_MEM.branch_cond = False

    def check_branch_condition(self, branch_type: BFunct3, zf, sf, of, cf) -> bool:
        """
        根据四个符号位判断跳转条件
        """
        if branch_type == BFunct3.BEQ:
            return zf == 1
        elif branch_type == BFunct3.BNE:
            return zf == 0
        elif branch_type == BFunct3.BLT:
            return sf ^ of == 1
        elif branch_type == BFunct3.BGE:
            return sf ^ of == 0
        elif branch_type == BFunct3.BLTU:
            return cf == 1
        elif branch_type == BFunct3.BGEU:
            return cf == 0
        else:
            raise ValueError(f"unknown branch type {branch_type}")

    def show_info(self, info=None):
        mem_addr_start = 0
        mem_range = 20
        mem_align = 4
        register_range = 32
        register_align = 8

        if info is not None:
            print(info)
        print("-" * 20)
        for i in range(mem_range // mem_align):
            for j in range(mem_align):
                index = i * mem_align + j
                print(
                    f"mem[{index + mem_addr_start:2}] = {self.memory[index + mem_addr_start]:3}",
                    end=" |",
                )
            print("")
        print("-" * 20)
        for i in range(register_range // register_align):
            for j in range(register_align):
                index = i * register_align + j
                print(f"r{index:<2} = {self.registers[index]:3}", end=" |")
            print("")
        print("-" * 20)

    def binary_str(self, imm: str):
        """
        取补码计算, 如果首位为 0 则直接计算值, 如果为 1 则 01 取反加一
        """
        if imm[0] == "1":
            inverted_str = "".join("1" if bit == "0" else "0" for bit in imm)
            abs_value = int(inverted_str, 2) + 1
            return -abs_value
        else:
            return int(imm, 2)


In [4]:
from instructions import *
from isa import *


def main():
    # basic loop -> loop.S
    #
    #     lui     a5,0x1
    #     addi    a5,a5,-100 # f9c <.L2+0xf90>
    #     add     a5,a0,a5
    #
    # L2:
    #     lw      a4,0(a5)
    #     nop
    #     add     a4,a4,a1
    #     nop
    #     nop
    #     sw      a4,0(a5)
    #     addi    a5,a5,-4
    #     nop
    #     bne     a0,a5, L2
    #     ret
    instructions = [
        0x000017B7,
        0xF9C78793,
        0x00F507B3,
        0x0007A703,
        0x00000013,
        0x00B70733,
        0x00000013,
        0x00000013,
        0x00E7A023,
        0xFFC78793,
        0x00000013,
        0xFEF510E3,
        0x00008067,
    ]

    isa = PipelineISA()
    isa.registers[10] = 0x200
    isa.registers[11] = 1
    isa.load_instructions(instructions)
    isa.run()
    basic_step = isa.step

    # schedule loop -> schedule_loop.S
    #
    #     lui     a5,0x1
    #     addi    a5,a5,-100 # f9c <.L2+0xf90>
    #     add     a5,a0,a5
    #
    # L2:
    #     lw      a4,0(a5)
    #     addi    a5,a5,-4
    #     add     a4,a4,a1
    #     nop
    #     bne     a0,a5, L2
    #     sw      a4,4(a5)
    #     ret
    schedule_instructions = [
        0x000017B7,
        0xF9C78793,
        0x00F507B3,
        0x0007A703,
        0xFFC78793,
        0x00B70733,
        0x00000013,
        0xFEF518E3,
        0x00E7A223,
        0x00008067,
    ]

    isa.reset()
    isa.registers[10] = 0x200
    isa.registers[11] = 1
    isa.load_instructions(schedule_instructions)
    isa.run()
    schedule_step = isa.step

    print(f"   basic step = {basic_step}")
    print(f"schedule step = {schedule_step}")
    print(
        f"   improvment = {basic_step}-{schedule_step}/{basic_step} = {(basic_step-schedule_step)/basic_step * 100:.2f}%"
    )


if __name__ == "__main__":
    main()


   basic step = 10996
schedule step = 7001
   improvment = 10996-7001/10996 = 36.33%
